# 1조 홍성진 Exploration 04

# 모듈 불러오기


In [1]:
import os, re
import numpy as np
import tensorflow as tf

import glob

## 데이터 읽어오기

In [2]:
txt_file_path = os.getenv("HOME") + '/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r", encoding='UTF8') as f: #cp949 에러가 나서 encoding='UTF8'를 추가해 주었다.
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[1290:1300])

데이터 크기: 187088
Examples:
 ['Yes and lover, lover, lover, lover, lover, lover, lover come back to me. Un Canadien Errant (A wandering Canadian,)', 'Banni de ses foyers, (banned from his hearths,)', 'Parcourait en pleurant (travelled while crying)', 'Des pays etrangers. (in foreign lands.)', 'Parcourait en pleurant (travelled while crying)', 'Des pays etrangers. (in foreign lands.)', 'Un jour, triste et pensif, (One day, sad and pensive,)', 'Assis au bord des flots, (sitting by the flowing waters,)', 'Au courant fugitif (to the fleeing current)', 'Il adressa ces mots: (he addressed these words:)']


## 데이터 정제

In [3]:
def preprocess_sentence(stc):
    stc = stc.lower().strip() #1. 소문자로 바꾸고, 양쪽 공백을 지웁니다.
    stc = re.sub(r"([?.!,¿])", r" \1 ",stc) #2. 특수문자 양쪽에 공백을 널습니다.
    stc = re.sub(r'[" "]+', " ", stc) # 3. 여러개의 공백은 하나의 공백으로 바꿉니다.
    stc = re.sub(r"[^a-zA-Z?.!,¿']+", " ",stc) # 4. a-z, A-z, ? ,, !, ¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다.
    stc = stc.strip() # 다시 양쪽 공백을 지웁니다.
    stc = '<start> ' + stc + ' <end>' # 문장 시작에는 <start>, 끝에는 <end>를 추가합니다.
    return stc

#문장 필터링이 잘 되는지 확인.
print(preprocess_sentence("(This @_is ;;;sample        sentence. isn't it?)"))

<start> this is sample sentence . isn't it ? <end>


In [4]:
corpus = [] # 정제된 문장을 이곳에 모으기

set_corpus = set(raw_corpus) #중복 제외
for sentence in set_corpus:
    #원하지 않는 문장 건너 뛰기.
    if len(sentence) == 0: continue
    # if sentence[-1] == ":": continue 대본이 아닌 가사에는 굳이 필요없음.
#     if sentence.count(" ")-(sentence.count("?")+sentence.count(".")+sentence.count("!")+sentence.count("¿")+sentence.count(",")) > 15:
#         #print("removed: \n",sentence,"DONE. \n") # 삭제된 문장을 잠시 확인하였음.
#         continue #토큰의 개수가 15개를 넘어가는 문장 학습데이터에서 제외
    
    
    #정제 후 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    word = sentence.split() #토큰이 15개가 넘어가면 학습 데이터에서 제외
    if len(word) > 15:
        #print(word)
        continue
    
    else:
        corpus.append(preprocessed_sentence)

#정제 결과 10개정도 확인
corpus[:10]

["<start> out of my life girl , you don't understand what you do to me <end>",
 '<start> mesmo a bela e a fera sentimento assim <end>',
 '<start> showing my ass , growing up then started traveling <end>',
 '<start> what am i supposed to do to make you want me properly ? <end>',
 '<start> the law can t touch her at all <end>',
 '<start> i spot hiphop in the ocean im gon save it <end>',
 "<start> what's your name ? <end>",
 '<start> it might just happen <end>',
 '<start> did they know right then and there what the power was worth ? <end>',
 '<start> and they tell him , <end>']

## Tokenize 하기

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000, #12000이상
        filters = '', #이미 했기 때문에 필요없음.
        oov_token = "<unk>" # num_words 에 포함되지 않으면 <unk>
    )
    
    tokenizer.fit_on_texts(corpus) #tokenizer 내부의 corpus를 이용한 단어장 완성
    
    tensor = tokenizer.texts_to_sequences(corpus) #tokenizer을 이용해 corpus를 Tensor로 변환
    
    #시퀀스가 짧다면, 문장 뒤나 앞에 패딩을 붙여 길이를 맞춰줍니다.
    #문장 뒤에 패딩을 맞추고 싶으면 padding = 'post'
    #문자 앞에 패딩을 맞추고 싶으면 padding = 'pre'
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

print("\n",'='*70,"\n",tensor[:3, :-1])

[[    2    55    17 ...     0     0     0]
 [    2 11382     9 ...     0     0     0]
 [    2  2723    12 ...     0     0     0]
 ...
 [    2    74    96 ...     0     0     0]
 [    2   203   260 ...     0     0     0]
 [    2    40    20 ...     0     0     0]] <keras_preprocessing.text.Tokenizer object at 0x7f52b366c400>

 [[    2    55    17    12   107    82     4     7    34   426    41     7
     44    10    11     3     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [    2 11382     9 11383   608     9  7680     1     1     3     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [    2  2723    12   208     4  1854    30    84   546  1984     3     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]]


## Tokenizer 사전 확인

In [17]:
print(len(tokenizer.index_word))
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx], end ='\t')
    
    if idx >= 20: break


28540
1 : <unk>	2 : <start>	3 : <end>	4 : ,	5 : the	6 : i	7 : you	8 : and	9 : a	10 : to	11 : me	12 : my	13 : in	14 : it	15 : .	16 : that	17 : of	18 : on	19 : your	20 : i'm	

## 평가 데이터 셋 소스와 타겟으로 분리 1

In [7]:
#tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
#마지막 토큰은 <end>가 아닌, <pad>일 가능성이 높다.

src_input = tensor[:,:-1]
#tensor에서 <start>를 잘라내서 타겟 문장을 생성
trg_input = tensor[:,1:]

print(src_input[0])
print(trg_input[0])

[  2  55  17  12 107  82   4   7  34 426  41   7  44  10  11   3   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[ 55  17  12 107  82   4   7  34 426  41   7  44  10  11   3   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


## 데이터셋 객체를 생성

In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여, 12001개.
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만듭니다.
# 데이터셋에 대해서는 아래 문서를 참고
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, trg_input)) #courpus텐서를 tf.data.Dataset 객체로 변환.
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)>

## 평가 데이터 셋 소스와 타겟으로 분리 2

In [9]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                         trg_input,
                                                         test_size = 0.2,
                                                         shuffle = True,
                                                         random_state  = 99)

In [10]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (89681, 32)
Target Train: (89681, 32)


## 인공지능 만들기

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 #워드벡터의 차원수. 즉, 단어가 추상적으로 표현되는 크기.
hidden_size = 1024 #얼마나 많은 일꾼을 둘 것인가? 같은데이터를 가지고 각자의 생각을 갖는다.
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
#데이터셋에서 데이터 한 배치만 불러온다.
for src_sample, trg_sample in dataset.take(1): break

lyricist(src_sample) # 한 배치만 불러온 데이터를 모델에 넣어봅니다.

<tf.Tensor: shape=(256, 32, 12001), dtype=float32, numpy=
array([[[-1.50271386e-04, -6.50537841e-05, -2.88962852e-04, ...,
          1.72114214e-05,  1.10579633e-04,  6.44221873e-05],
        [-1.94001492e-04,  1.27035382e-04, -3.29822738e-04, ...,
         -9.44565545e-05,  2.18683868e-04,  2.42243405e-04],
        [-2.72103149e-04,  3.15279554e-04, -2.83152127e-04, ...,
         -2.94811936e-04,  3.16738558e-04,  2.22071219e-04],
        ...,
        [ 2.37131972e-04, -2.50920653e-03,  8.02296796e-04, ...,
         -5.00246603e-03, -4.43833051e-06, -8.32079723e-03],
        [ 1.94075605e-04, -2.64868955e-03,  7.80237315e-04, ...,
         -5.08231716e-03,  7.12486290e-05, -8.44028220e-03],
        [ 1.60147843e-04, -2.77428911e-03,  7.57639413e-04, ...,
         -5.14460262e-03,  1.40535194e-04, -8.53389408e-03]],

       [[-1.50271386e-04, -6.50537841e-05, -2.88962852e-04, ...,
          1.72114214e-05,  1.10579633e-04,  6.44221873e-05],
        [-1.10269430e-05, -3.06040311e-04, -6

In [13]:
lyricist.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [14]:
#Loss
# 알고싶으면 아래 문서 참조
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

lyricist.compile(loss = loss, optimizer = optimizer)
lyricist.fit(dataset, epochs = 5)

Epoch 1/5
437/437 [==============================] - 164s 370ms/step - loss: 1.7903
Epoch 2/5
437/437 [==============================] - 176s 402ms/step - loss: 1.4976
Epoch 3/5
437/437 [==============================] - 177s 405ms/step - loss: 1.4276
Epoch 4/5
437/437 [==============================] - 177s 404ms/step - loss: 1.3777
Epoch 5/5
437/437 [==============================] - 177s 404ms/step - loss: 1.3376


In [15]:
def generate_text(lyricist, tokenizer, init_sentence="<start>", max_len=32):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = lyricist(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [16]:
generate_text(lyricist, tokenizer, init_sentence="<start> I love ", max_len=32)

"<start> i love you , i don't know what i do <end> "

# 회고

    이번 프로젝트를 하며 가장 어려웠던 점은, enc_train.shape 과, dec_train.shape 을 맞추는 것이었습니다.
    처음 노드를 보았을 때, 각각 Source Train: (124960,14), Target Train: (124960,14)를 얻도록 검토해 보라고 해서, 그 부분에서 많은 갈등이 있었습니다. 후에 노드가 바뀌어 그 구간이 사라졌지만, 어쨋든 저는 혹시 그 부분을 줄이거나 늘릴 수 있다면, 어떤방법이 있을까 고민을 많이하였습니다. 앞 부분을 표시하는 124960 은, 중간에 학습 문장수를 줄이면서 같이 줄었지만, 뒤의 14부분은 줄이는데에 실패하였습니다. 하지만 조사를 하며 알아낸 점이 있기에 절대 시간낭비라는 생각은 하지 않습니다. 우선, 노드 마지막 부분에 말씀해주신 14라는 부분은 노드 4-5. 부분에서 알아낼 수 있었습니다. max_len가 14로 맞춰 져 있었다는 말일텐데, 저는 그 부분이 32로 나왔었습니다. 안그래도, 패딩후에 실험삼아 tensor의 개수를 세 보았는데 32로 일치하였고, 이부분이 같다는 점을 알아 내었습니다.

    이번 노드는 제게 흥미로운 부분들이 많았기에, 다양한 실험들을 했다고 할 수 있는데, 그로인해 모호한 점이 많았습니다.
    먼저, 위에 말씀드린 max_len를 직접 조작할 수 있는지가 궁금하였고, 그로 인한 따라오는 궁금증 으로는, 왜 32로 고정되었을까? 입니다. 처음 생각했을 때에는 토큰의 개수가 15개가 넘어가는 문장은 학습데이터에서 제외했으므로, 패딩을 한 후의 tensor 역시 최대 15개 일 줄았으나, 그러지 않았다는점이 궁금하였습니다.
    다음으로 궁금한 점은, 단어장의 크기입니다. 분명 단어장의 크기를 12000으로 조절하였음에도, 패딩을 포함하여 12001임에도, 토큰의 총 개수가 28540개로 나오는 점이 궁금하였습니다.
    마지막으로 아직까지 해결하지 못한 문제는, 마지막 output에 <unk>이 섞여서 나온다는 점입니다. <unk>은 단어장 12000개에 포함되지 않는 단어임은 알고있습니다. 하지만 왜 이 단어가 마지막 테스트 하는 부분에서 끼어서 나올까가 궁금했습니다. 사실 마지막 부분에 generate_tex(model, tokenizer .... ) 이런식으로 썼었는데, 노드 마지막에서 제공해 준 대로 generate_text(lyricist, ....) 로 model에서 lyricist로 바꾸어 주었는데, 그랬더니 <unk> 현상은 사라지는대신, 같은 단어를 여러번 반복하는 또 다른 의문점을 만드는 일이 생기고 말았었습니다. 다시 고쳐지기는 했으나, 어떤 원인으로인해 생인 현상인지를 아직 찾지 못하였다는 점이 참으로 슬픕니다...

    이번 루브릭 지표를 맞추기 위해 시도한 것들로는 결과에 <unk>같은 문장이 나오지 않게 하기 위해 노력하였고, 결과 값에 계속 같은 패턴의 단어를 반복적으로 사용하는것이, '노래 가사' 라는 특성상, 같은 가사가 두번 이상 반복되는 경우가 많을 것 같아서, splitline으로 나누어 둔 줄 가사 list를 set으로 바꿔 줌 으로써 중복 list를 없애버려 학습에 도움이 되도록 해 보았습니다. 또한, don't, i'm, it's 등의 ' 특수문자가 있어서 기본 노드에서 가져 온 특수문자 제서 파트에 '가 정상적으로 출력이 되도록 만들었습니다. 토큰의 개수가 15개를 넘지 않게 하기 위해, split이라는 함수를 사용하여, sentence내의 문장을 우선 띄어쓰기를 기준으로 잘라내어 15 단어가 넘으면, 토큰화에서 제거하였습니다. validation loss 값이 2.2 이하로 떨어지게 하기 위하여 embeddig size 와 hidden size 등을 조정해 보았습니다.
    만약 루브릭 평가 관련 지표를 달성하지 못하였다면, 위에 아직 알아내지 못한 max_len를 조절할 수 있는지가 아직 의문이 남은 점, 그리고 마지막 결과 가사가 매끄럽지 못한 부분들이 루브릭 관련 지표를 달성하지 못하게 만들지 않았나 고려해 봅니다.
    
감사합니다.